<a href="https://colab.research.google.com/github/kryuchkovdm/Distillation/blob/master/models/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, n_vocabs, embed_dims, n_lstm_units, n_lstm_layers, n_output_classes,special_chars=[], use_dropout=True):
        super(BiLSTM, self).__init__()
        self.v = n_vocabs
        self.e = embed_dims
        self.u = n_lstm_units
        self.l = n_lstm_layers
        self.o = n_output_classes

        self.embed = nn.Embedding(
            self.v,
            self.e
            )
        self.bilstm = nn.LSTM(
            input_size = self.e,
            hidden_size = self.u,
            num_layers = self.l,
            batch_first = True,
            bidirectional = True,
            dropout=0.5 if use_dropout else 0.
        )
        self.dropout = nn.Dropout(0.5 if use_dropout else 0.)
        self.linear = nn.Linear(
            self.u * 2,
            self.o
        )
        for special in special_chars:
            self.embed.weight.data[special] = torch.zeros(embed_dims)
    def forward(self, X):
        h0 = torch.zeros(self.l * 2, X.size(0), self.u).to(device)
        c0 = torch.zeros(self.l * 2, X.size(0), self.u).to(device)

        out = self.embed(X)
        out, _ = self.bilstm(out, (h0, c0))
        out, _ = torch.max(out, 1)
        out = self.dropout(out)
        out = self.linear(out)
                
        return out